In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)


# prepare the model input
prompt = "中国首都是哪?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
text

'<|im_start|>user\n中国首都是哪?<|im_end|>\n<|im_start|>assistant\n'

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
model

[2025-04-30 13:37:12,275] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/samtang/miniconda3/envs/rl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/samtang/miniconda3/envs/rl

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [30]:
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    enable_thinking=False, # Switches between thinking and non-thinking modes. Default is True.
)
text

[151644,
 872,
 198,
 58695,
 59975,
 100132,
 99459,
 30,
 151645,
 198,
 151644,
 77091,
 198,
 151667,
 271,
 151668,
 271]

In [31]:
torch.Tensor(text).long().unsqueeze(0)

tensor([[151644,    872,    198,  58695,  59975, 100132,  99459,     30, 151645,
            198, 151644,  77091,    198, 151667,    271, 151668,    271]])

In [32]:
import torch
outputs = model.generate(input_ids=torch.Tensor(text).long().unsqueeze(0).to('cuda'), max_length=512)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'user\n中国首都是哪?\nassistant\n<think>\n\n</think>\n\n中国首都是**北京**。'